In [3]:
import random
import copy
import datetime
from dataclasses import dataclass
from typing import List, Dict, Tuple, Sequence, Callable

In [105]:
@dataclass
class Chromosome:
    genes   : List
    fitness : float


In [72]:
def generate_parent(length: int, geneSet: Sequence, get_fitness: Callable)->Chromosome:
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)


In [84]:
geneSet = [i for i in range(100)]

In [85]:
def dummy(genes):
    return 1

In [86]:
genes = generate_parent(10, geneSet, dummy)

In [87]:
genes

Chromosome(genes=[7, 36, 95, 87, 3, 18, 62, 52, 75, 80], fitness=1)

In [95]:
def get_fitness(genes: Sequence)->float:
    fitness = 1
    
    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            fitness += 1
    return fitness


In [98]:
genes = generate_parent(10, geneSet, get_fitness)

In [99]:
genes

Chromosome(genes=[43, 48, 51, 12, 34, 8, 50, 28, 14, 41], fitness=6)

In [103]:
def test_get_fitness_gives_correct_values():
    
    for r in [6,11,101]:
        mygenes = [i for i in range(1,r)]
    
        if get_fitness(mygenes) == len(mygenes):
            print(f'Hector no te enteras')
        else:
            print(f'Daddy estas viejo')

In [104]:
test_get_fitness_gives_correct_values()

Hector no te enteras
Hector no te enteras
Hector no te enteras


In [106]:
def get_fitness(genes: Sequence)->float:
    fitness = 1
    
    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            fitness += 1
    return fitness


def mutate_genes(genes: List, geneSet: List)->List:
    """Mutate genes target choosing character from geneSet"""

    mutatedGenes         = genes[:]
    index                = random.randrange(0, len(mutatedGenes))
    newGene, alternate   = random.sample(geneSet, 2)
    mutatedGenes[index]  = alternate if newGene == mutatedGenes[index] else newGene
    return mutatedGenes


def mutate_chromosome(geneSet:     Sequence,
                      parent:      Chromosome,
                      get_fitness: Callable)->Chromosome:
    """Generate a mutated chromosome"""
    mutatedGenes = mutate_genes(parent.genes, geneSet)
    fitness      = get_fitness(mutatedGenes)
    return Chromosome(mutatedGenes, fitness)



In [112]:
geneSet = [i for i in range(100)]

def sort_numbers(totalNumbers, imax=1000):
    startTime = datetime.datetime.now()
    random.seed()
    setToSort = generate_parent(totalNumbers, geneSet, get_fitness)
    print(f'Set to Sort = {setToSort}')
    
    optimalFitness = len(setToSort.genes)
    print(f'optimal fitness = {optimalFitness}')
    
    i =0
    while setToSort.fitness < optimalFitness and i < imax:
        mutateChromosome = mutate_chromosome(geneSet, setToSort, get_fitness)
        if mutateChromosome.fitness > setToSort.fitness:
            setToSort = mutateChromosome
            print(f'new guess increased fitness : {setToSort}, i = {i}')
        i+=1
    print(setToSort)
    print(i)




In [114]:
sort_numbers(totalNumbers=10, imax=10000)

Set to Sort = Chromosome(genes=[86, 58, 65, 71, 16, 34, 18, 41, 9, 22], fitness=6)
optimal fitness = 10
new guess increased fitness : Chromosome(genes=[50, 58, 65, 71, 16, 34, 18, 41, 9, 22], fitness=7), i = 42
new guess increased fitness : Chromosome(genes=[50, 58, 65, 71, 16, 34, 37, 41, 9, 22], fitness=8), i = 61
Chromosome(genes=[50, 58, 65, 71, 16, 34, 37, 41, 9, 22], fitness=8)
10000


In [115]:
@dataclass
class Fitness:
    sortCount : int
    gap       : int

@dataclass
class Chromosome:
    genes   : List
    fitness : Fitness


In [116]:
def get_fitness(genes : List)->Fitness:
    sortCount = 1
    gap = 0

    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            sortCount += 1
        else:
            gap += genes[i - 1] - genes[i]
    return Fitness(sortCount, gap)


In [123]:
def compare_fitness(f1: Fitness, f2: Fitness)->bool:
    """Returns True if f1 is fitter than f2"""
    if f1.sortCount == f2.sortCount:
        return f1.gap <=f2.gap
    else: 
        return f1.sortCount >= f2.sortCount

In [126]:
geneSet = [i for i in range(100)]

def sort_numbers(totalNumbers, verbose=True, imax=10000):
    startTime = datetime.datetime.now()
    random.seed()
    setToSort = generate_parent(totalNumbers, geneSet, get_fitness)
    print(f'Set to Sort = {setToSort}')

    optimalFitness = Fitness(totalNumbers, 0)
    
    i = 0
    while compare_fitness(setToSort.fitness, optimalFitness) == False and i < imax:
        newGuess = mutate_chromosome(geneSet, setToSort, get_fitness)
        if compare_fitness(newGuess.fitness, setToSort.fitness) == True:
            if verbose:
                print(f'new guess increased fitness  (increased sortCount): {setToSort}, i = {i}')
            setToSort = Chromosome(newGuess.genes, newGuess.fitness)
        
        i+=1
    print(i)
    print(setToSort)
    


 

In [127]:
sort_numbers(10)

Set to Sort = Chromosome(genes=[64, 99, 41, 69, 68, 5, 66, 38, 43, 32], fitness=Fitness(sortCount=5, gap=161))
new guess increased fitness  (increased sortCount): Chromosome(genes=[64, 99, 41, 69, 68, 5, 66, 38, 43, 32], fitness=Fitness(sortCount=5, gap=161)), i = 0
new guess increased fitness  (increased sortCount): Chromosome(genes=[96, 99, 41, 69, 68, 5, 66, 38, 43, 32], fitness=Fitness(sortCount=5, gap=161)), i = 2
new guess increased fitness  (increased sortCount): Chromosome(genes=[96, 99, 41, 69, 68, 5, 66, 38, 43, 81], fitness=Fitness(sortCount=6, gap=150)), i = 3
new guess increased fitness  (increased sortCount): Chromosome(genes=[33, 99, 41, 69, 68, 5, 66, 38, 43, 81], fitness=Fitness(sortCount=6, gap=150)), i = 4
new guess increased fitness  (increased sortCount): Chromosome(genes=[33, 99, 41, 69, 68, 5, 66, 38, 43, 92], fitness=Fitness(sortCount=6, gap=150)), i = 6
new guess increased fitness  (increased sortCount): Chromosome(genes=[33, 99, 41, 69, 30, 5, 66, 38, 43, 92],